In [1]:
# Importing the libraries
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import torch

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
full_dataset = datasets.ImageFolder('/kaggle/input/dataset/mammals', transform=transform)
print(len(full_dataset))

4500


In [3]:
test_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - test_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
vgg16 = models.vgg16(pretrained=False)

num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = torch.nn.Linear(num_features, 30)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

In [8]:
epochs = 10

for epoch in range(epochs):
    vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()  #
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

    # Test loop
    vgg16.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Loss: {test_loss/len(test_loader)}, Accuracy: {100 * correct / total}%')

Epoch 1/10, Loss: 2.6637969818790403
Test Loss: 2.8254833879141974, Accuracy: 19.22222222222222%
Epoch 2/10, Loss: 2.600430410520165
Test Loss: 2.7239640992263268, Accuracy: 22.444444444444443%
Epoch 3/10, Loss: 2.5486595060973043
Test Loss: 2.7563120989963927, Accuracy: 24.77777777777778%
Epoch 4/10, Loss: 2.5091292056362184
Test Loss: 2.671094540891976, Accuracy: 24.666666666666668%
Epoch 5/10, Loss: 2.435109102620488
Test Loss: 2.625076885881095, Accuracy: 25.444444444444443%
Epoch 6/10, Loss: 2.3550534121758115
Test Loss: 2.6501251418015053, Accuracy: 27.555555555555557%
Epoch 7/10, Loss: 2.2608301069884176
Test Loss: 2.620625479467984, Accuracy: 28.88888888888889%
Epoch 8/10, Loss: 2.199280025684728
Test Loss: 2.6632238749800057, Accuracy: 27.11111111111111%
Epoch 9/10, Loss: 2.1200918697677884
Test Loss: 2.6280855885867416, Accuracy: 29.0%
Epoch 10/10, Loss: 1.9838630741676397
Test Loss: 2.6288167608195336, Accuracy: 26.555555555555557%
